In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class VariableSizeCNN(nn.Module):
    def __init__(self, max_grid_size=50, latent_dim=128, num_channels=16):
        super(VariableSizeCNN, self).__init__()
        self.max_grid_size = max_grid_size
        self.latent_dim = latent_dim
        self.num_channels = num_channels

        # Encoder (CNN)
        self.conv1 = nn.Conv2d(1, num_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(num_channels, num_channels*2, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(num_channels*2, num_channels*4, kernel_size=3, padding=1)
        
        # Global Average Pooling to get fixed-size latent representation
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        
        # Fully connected layer to generate latent representation
        self.fc1 = nn.Linear(num_channels*4, latent_dim)
        
        # Decoder (Fully connected layers to generate output parameters)
        self.fc2 = nn.Linear(latent_dim, max_grid_size * max_grid_size)
        self.fc3 = nn.Linear(latent_dim, 10)  # Assuming max 10 unique values in output

    def forward(self, x):
        # Get input size
        _, _, h, w = x.size()
        
        # Encoder
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        
        # Global Average Pooling
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        
        # Generate latent representation
        latent = self.fc1(x)
        
        # Decoder
        output_values = torch.sigmoid(self.fc3(latent))  # Sigmoid to keep values between 0 and 1
        output_mask = self.fc2(latent).view(-1, 1, self.max_grid_size, self.max_grid_size)
        output_mask = output_mask[:, :, :h, :w]  # Crop to input size
        
        # Generate output
        output = torch.matmul(output_mask.view(-1, h*w, 1), output_values.unsqueeze(1))
        output = output.view(-1, 1, h, w)
        
        return output

# Usage example
model = VariableSizeCNN()

# Example input (batch_size=1, channels=1, height=10, width=10)
x = torch.randn(1, 1, 10, 10)

output = model(x)
print("Input shape:", x.shape)
print("Output shape:", output.shape)